In [15]:
import numpy as np
from scipy.stats import norm
import importlib
import helper_functions
importlib.reload(helper_functions)
from helper_functions import get_option_info

In [16]:
def mc_european(S0, K, r, sigma, T, N, call=True, seed=None, use_antithetic=False, use_control_variate=False):
    rng = np.random.default_rng(seed)
    M = N
    if use_antithetic:
        # we'll generate N/2 normals and mirror them
        if N % 2 != 0:
            raise ValueError("N must be even when using antithetic variates")
        M = N // 2
    # draw normals
    Z = rng.standard_normal(M)
    if use_antithetic:
        Z_full = np.concatenate([Z, -Z])
    else:
        Z_full = Z

    # terminal prices
    drift = (r - 0.5 * sigma**2) * T
    vol = sigma * np.sqrt(T)
    S_T = S0 * np.exp(drift + vol * Z_full)

    # payoffs
    if call:
        Y = np.maximum(S_T - K, 0.0)   # undiscounted payoff
    else:
        Y = np.maximum(K - S_T, 0.0)

    # control variate using S_T (its expectation under Q is S0*exp(rT))
    if use_control_variate:
        X = S_T
        EX = S0 * np.exp(r * T)
        # estimate optimal b
        cov = np.cov(Y, X, bias=False)[0, 1]
        varX = np.var(X, ddof=1)
        b_hat = cov / varX if varX > 0 else 0.0
        Y_adj = Y - b_hat * (X - EX)
        price = np.exp(-r * T) * np.mean(Y_adj)
        se = np.exp(-r * T) * np.std(Y_adj, ddof=1) / np.sqrt(len(Y_adj))
        return price, se, {'b_hat': b_hat}
    else:
        price = np.exp(-r * T) * np.mean(Y)
        se = np.exp(-r * T) * np.std(Y, ddof=1) / np.sqrt(len(Y))
        return price, se, {}


In [17]:
api="<your api key here>"
S0,K,r,sigma,T,mid_price,bid,ask=get_option_info("AAPL",api,True)

In [18]:
N=200000

In [19]:
mc_price1,se1,dic1=mc_european(S0,K,r,sigma,T,N,call=True,use_antithetic=False,use_control_variate=False)
mc_price2,se2,dic2=mc_european(S0,K,r,sigma,T,N,call=True,use_antithetic=True,use_control_variate=False)
mc_price3,se3,dic3=mc_european(S0,K,r,sigma,T,N,call=True,use_antithetic=True,use_control_variate=True)

In [20]:
if mc_price1>=bid and mc_price1<=ask:
    print("Black-Scholes-Merton model is within range of market bid and ask")
else:
    print("Black-Scholes-Merton model is not within the range of market bid and ask")
print("bid: ",bid)
print("ask: ",ask)
print("mid (model price should be close to this): ",mid_price)
print("model price: ",mc_price1)

Black-Scholes-Merton model is within range of market bid and ask
bid:  162.5
ask:  165.1
mid (model price should be close to this):  163.8
model price:  163.23060791917945


In [21]:
if mc_price2>=bid and mc_price2<=ask:
    print("Black-Scholes-Merton model is within range of market bid and ask")
else:
    print("Black-Scholes-Merton model is not within the range of market bid and ask")
print("bid: ",bid)
print("ask: ",ask)
print("mid (model price should be close to this): ",mid_price)
print("model price: ",mc_price2)

Black-Scholes-Merton model is within range of market bid and ask
bid:  162.5
ask:  165.1
mid (model price should be close to this):  163.8
model price:  163.23697728228723


In [22]:
if mc_price3>=bid and mc_price3<=ask:
    print("Black-Scholes-Merton model is within range of market bid and ask")
else:
    print("Black-Scholes-Merton model is not within the range of market bid and ask")
print("bid: ",bid)
print("ask: ",ask)
print("mid (model price should be close to this): ",mid_price)
print("model price: ",mc_price3)

Black-Scholes-Merton model is within range of market bid and ask
bid:  162.5
ask:  165.1
mid (model price should be close to this):  163.8
model price:  163.23709037120327
